# A simple UCC computation: $H_2$ molecule

This is a minimal example of how to perform a chemistry computation with the QLM. The molecule is $H_2$. For a more advanced example (with active space selection, noisy simulation, etc, see the notebook on LiH).

In [1]:
import numpy as np

from qat.fermion.chemistry.pyscf_tools import perform_pyscf_computation

# H2
geometry = [("H", (0.0, 0.0, 0.0)), ("H", (0.0, 0.0, 0.7414))]
basis = "sto-3g"
spin = 0  # = 2 S with S total spin angular momentum = # of unpaired electrons
charge = 0

(
    rdm1,
    orbital_energies,
    nuclear_repulsion,
    n_electrons,
    one_body_integrals,
    two_body_integrals,
    info,
) = perform_pyscf_computation(geometry=geometry, basis=basis, spin=spin, charge=charge, verbose=True, run_FCI=False)

print("Number of qubits = ", rdm1.shape[0] * 2)



from qat.fermion.chemistry import MolecularHamiltonian, MoleculeInfo

# Define the molecular hamiltonian
mol_h = MolecularHamiltonian(one_body_integrals, two_body_integrals, nuclear_repulsion)

print(mol_h)

##############################################################
# NOONs part: we want to avoid this part, i.e stay in MO basis
##############################################################
# Compute the natural orbitals occupation numbers and the basis transformation matrix
noons, basis_change = np.linalg.eigh(rdm1)
# The noons should be in decreasing order
# Since we reversed the noons, we have to flip the basis as well
noons = list(reversed(noons))
basis_change = np.flip(basis_change, axis=1)
mol_h_new_basis = mol_h.transform_basis(basis_change)

molecule = MoleculeInfo(
    mol_h_new_basis,
    n_electrons=n_electrons,
    noons=noons,
    orbital_energies=orbital_energies,
)
### end of NOONs part ########################################

from qat.fermion.chemistry.ucc import guess_init_params, get_hf_ket, get_cluster_ops

# Computation of the initial parameters
theta_init = guess_init_params(
    molecule.two_body_integrals,
    molecule.n_electrons,
    molecule.noons, # should not be needed
    molecule.orbital_energies,
)

# Define the initial Hartree-Fock state
ket_hf_init = get_hf_ket(molecule.n_electrons, molecule.noons) # noons not needed

# Compute the cluster operators
cluster_ops = get_cluster_ops(molecule.n_electrons, molecule.noons) # noons not needed

###### transformation from fermion to qubits ###################

from qat.fermion.transforms import transform_to_jw_basis
from qat.fermion.transforms import recode_integer, get_jw_code

# Compute the ElectronicStructureHamiltonian
H = molecule.hamiltonian.get_electronic_hamiltonian()

# Transform the ElectronicStructureHamiltonian into a spin Hamiltonian
H_sp = transform_to_jw_basis(H)

# Express the cluster operator in spin terms
cluster_ops_sp = [transform_to_jw_basis(t_o) for t_o in cluster_ops]

# Encoding the initial state to new encoding
hf_init_sp = recode_integer(ket_hf_init, get_jw_code(H_sp.nbqbits))

from qat.lang.AQASM import Program, X
from qat.fermion.trotterisation import make_trotterisation_routine

prog = Program()
reg = prog.qalloc(H_sp.nbqbits)

# Initialize the Hartree-Fock state into the Program
for j, char in enumerate(format(hf_init_sp, "0" + str(H_sp.nbqbits) + "b")):
    if char == "1":
        prog.apply(X, reg[j])

# Define the parameters to optimize
theta_list = [prog.new_var(float, "\\theta_{%s}" % i) for i in range(len(cluster_ops))]

# Define the parameterized Hamiltonian
cluster_op = sum([theta * T for theta, T in zip(theta_list, cluster_ops_sp)])

# Trotterize the Hamiltonian (with 1 trotter step)
qrout = make_trotterisation_routine(cluster_op, n_trotter_steps=1, final_time=1)

prog.apply(qrout, reg)
circ = prog.to_circ()


/usr/local/lib64/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


HF energy= -1.1166843870853405
MP2 energy= -1.1298551535553094
FCI energy= None
Number of qubits =  4
 MolecularHamiltonian(
 - constant_coeff : 0.7137539936876182
 - integrals shape
    * one_body_integrals : (2, 2)
    * two_body_integrals : (2, 2, 2, 2)



In [2]:
job = circ.to_job(observable=H_sp, nbshots=0)

from qat.qpus import LinAlg
from qat.plugins import ScipyMinimizePlugin

optimizer_scipy = ScipyMinimizePlugin(method="COBYLA", tol=1e-3, options={"maxiter": 1000}, x0=theta_init)
qpu = optimizer_scipy | LinAlg()
result = qpu.submit(job)

print("Minimum energy =", result.value)
print("FCI energy:", info["FCI"])

Minimum energy = -1.1372692374939386
FCI energy: None
